In [1]:
import os

In [2]:
%pwd

'd:\\projects\\Data-Extraction-and-NLP\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\projects\\Data-Extraction-and-NLP'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    # source_URL: str
    # local_data_file: Path
    # unzip_dir: Path

In [19]:
from src.DataExtractionAndNLP.constants import *
from src.DataExtractionAndNLP.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        # schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        # self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir
            # source_URL=config.source_URL,
            # local_data_file=config.local_data_file,
            # unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [24]:
from src.DataExtractionAndNLP import logger
from src.DataExtractionAndNLP.utils.common import get_size
import pandas as pd
import requests
from bs4 import BeautifulSoup

: 

In [ ]:
# Read the URLs and URL IDs from Input.xlsx
# input_file = 'Input.xlsx'
# df = pd.read_excel(input_file)

# Create a directory to save the extracted articles
output_dir = 'extracted_articles'
os.makedirs(output_dir, exist_ok=True)

for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']

    try:
        # Fetch the webpage content
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'})
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the article title and text 
        title = soup.select_one('h1.entry-title').get_text()
        article_text = soup.select_one('div.td-post-content.tagdiv-type').get_text()

        # Save the extracted article to a text file
        output_file = os.path.join(output_dir, f'{url_id}.txt')
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(f"{title}\n")
            f.write(article_text)

        print(f"Extracted article from {url} and saved to {output_file}")
    except Exception as e:
        print(f"Error processing {url}: {e}")

print("Extraction completed. Check the 'extracted_articles' directory for the saved files.")

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e